# A very basic quantum circuit

In [1]:
# Imports
import pennylane as qml
# Pennylane uses a special version of numpy which is optimized for quantum objects, and so we need to use their version
from pennylane import numpy as np

# Create a quantum device

In [2]:
#  Here we create a quantum device. Pennylane being a plug and play module, we can plug in any quantum computer in it
# By default we use the in built simulator that Pennylane provides. The wires are basically the number of qubits used.
dev = qml.device('default.qubit', wires=1)

In [3]:
# We need to wrap our circuit function with the qnode decorator in order to tell Pennylane that this is a quantum object.
@qml.qnode(dev)
def circuit(params):
    # Our circuit is very simple. It takes in a list of two parameters - theta1 and theta2.
    # These are the angles with which our qubit is rotated by about the X and Y axes respectively.
    # The wires parameter here specifies which qubit we want to rotate. Since we have only 1 qubit, wires=0
    qml.RX(params[0],wires=0)
    qml.RY(params[1],wires=0)
    # We then measure our qubit with respect to the Z axis. A measurement along σ_z involves sandwiching the sigma-z matrix between the qubit
    # vector and its complex conjugate. That is <Ψ|σ_z|Ψ>.
    return qml.expval(qml.PauliZ(0))

In [4]:
# Just to test out our circuit, let's see what the expected value is for [0,0.5]
params = [0,0.5]
circuit(params)

tensor(0.87758256, requires_grad=True)

In [5]:
# This is our ML cost function. This function measures the quality of our output against the ground truth.
# Our aim is to minimize this cost function. A high cost means a worse result.
# For the sake of this simple example, let's set our cost function to our circuit itself. It will be the aim of gradient descent to
# minimize just the output.
def cost(x):
    return circuit(x)

In [6]:
# We set our initial parameters like so. The initial cost is 0.87227
init_params = np.array([0.11,0.5])
cost(init_params)

tensor(0.87227854, requires_grad=True)

In [7]:
# We shall use gradient descent to optimize our input. Here GD will try to search for the values of X-rotation (theta1) and Y-rotation
# (theta2) that will minimize the value of measurement about the Z-axis. The stepsize is the learning rate.
opt = qml.GradientDescentOptimizer(stepsize=0.4)
steps = 100
params = init_params

In [8]:
for i in range(steps):
    params = opt.step(cost,params)
    if (i+1)%5==0:
        print("Cost after step {:5d} : {: .7f}".format(i+1,cost(params)))
print("Optimized rotation angles: {}".format(params))

Cost after step     5 : -0.4151640
Cost after step    10 : -0.9926820
Cost after step    15 : -0.9999554
Cost after step    20 : -0.9999997
Cost after step    25 : -1.0000000
Cost after step    30 : -1.0000000
Cost after step    35 : -1.0000000
Cost after step    40 : -1.0000000
Cost after step    45 : -1.0000000
Cost after step    50 : -1.0000000
Cost after step    55 : -1.0000000
Cost after step    60 : -1.0000000
Cost after step    65 : -1.0000000
Cost after step    70 : -1.0000000
Cost after step    75 : -1.0000000
Cost after step    80 : -1.0000000
Cost after step    85 : -1.0000000
Cost after step    90 : -1.0000000
Cost after step    95 : -1.0000000
Cost after step   100 : -1.0000000
Optimized rotation angles: [4.16641834e-22 3.14159265e+00]


In [9]:
# We can see that the result makes perfect sense. In order to achieve a minimum Z-value, we must not rotate about the X-axis (hence 
# a very small rotation value), and we must rotate about Y-axis by Π degrees, to get a minimum (-1) Z value.